#Установка и мпорт библиотек

In [ ]:
!pip install nltk
!pip install langchain
!pip install spacy
!python -m spacy download ru_core_news_sm  # Загрузка модели для русского языка в spacy

In [2]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Загрузка стоп-слов
nltk.download('punkt')     # Загрузка списка знаков пунктуации
nltk.download('stopwords') # Загрузка списка стоп-слов для различных языков

import spacy
# Загрузка русского лемматизатора из SpaCy
nlp = spacy.load("ru_core_news_sm")

import re  # Модуль для работы с регулярными выражениями

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity         # Импорт косинусного расстояния

import langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter  # Импорт RecursiveCharacterTextSplitter из langchain для деления на chunks

import torch  # Импорт PyTorch

import gensim.downloader as api
# Загрузка модели GloVe (Global Vectors for Word Representation)
model = api.load("glove-wiki-gigaword-100")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


[==================================================] 100.0% 128.1/128.1MB downloaded


#Загружаем данные и словарь с аббревиатурами

In [4]:
df = pd.read_csv('all_df.csv') #данные
df.head()

,filename,paragraphs,department
0,Топливная надбавка_топливный сбор в ПЭ_ЭР,С 25.03.2024 г. во всех печатных формах ПЭ/ЭР ...,DRK
1,Неоформленные грузы,"Проверьте наличие заявки на забор груза, незав...",DRK
2,Выдача грузов по СМС при доставке,Выдача груза по СМС при доставке возможна в сл...,DRK
3,Клиент просит отключить уведомления (E-mail; В...,"\nДля отключения уведомлений, необходимо уточн...",DRK
4,Интернет-Эквайринг (оплата банковской картои...,\nНа главной странице сайта www.pecom.ru клиен...,DRK


In [3]:
abr = pd.read_csv('abbr.csv') #аббревиатуры
abr.columns = ['abbr', 'full'] # Переименование столбцов для удобства доступа
abr = abr.set_index('abbr')['full'].to_dict() # Преобразование в словарь для последующего использования при обучении модели

# Добавление дополнительной аббревиатуры в словарь
abr['FTL'] = 'Full Truck Load'

## Разделим статьи по отделам

In [5]:
df['department'].unique()

array(['DRK', 'FTL', 'OTR', 'CMK', 'FT'], dtype=object)

In [6]:
df_DRK = df[df['department'] == 'DRK']
df_CMK = df[df['department'] == 'CMK']
df_FTL = df[df['department'] == 'FTL']
df_OTR = df[df['department'] == 'OTR']
df_FT = df[df['department'] == 'FT']

#Пропишем необходимые функции для работы

## Предобработка данных

###Функция очистки текста

In [7]:
def clean_text(text):
    # Удаление специальных символов с помощью регулярного выражения
    text = re.sub(r'\W+', ' ', text)
    # Приведение текста к нижнему регистру
    text = text.lower()
    # Лемматизация текста с использованием SpaCy
    doc = nlp(text)
    # Сбор лемматизированных токенов, исключая стоп-слова
    lemmatized_text = ' '.join(token.lemma_ for token in doc if not token.is_stop)
    return lemmatized_text

###Функция пополнения словаря модели синонимами для часто используемых слов

In [8]:
synonyms_cache = abr  # Сразу добавляем аббревиатуры в кэш

def get_synonyms(word):
    if word not in synonyms_cache:  # Проверяем, есть ли слово в кэше синонимов
        try:
            sim_words = model.similar_by_word(word)  # Поиск схожих слов для данного слова в модели GloVe
            synonyms_cache[word] = [word for word, _ in sim_words][:1]  # Сохраняем только первый найденный синоним
        except KeyError:
            synonyms_cache[word] = []
    return synonyms_cache[word]  # Возвращаем список синонимов для данного слова из кэша

###Обогащение текста (контекста и запроса) синонимами

In [9]:
def expand_with_synonyms(text):
    tokenized = nlp(text)  # Токенизация текста с помощью SpaCy
    expanded_text = []  #список для расширенного текста
    for token in tokenized:
        expanded_text.append(token.text)  # Добавление оригинального токена в расширенный текст
        if token.has_vector:
            synonyms = get_synonyms(token.text)  # Получение синонимов для текущего токена
            if synonyms:
                expanded_text.append(synonyms[0])  # Добавление первого найденного синонима
    return ' '.join(expanded_text)  # Возврат расширенного текста в виде строки

### Разбиваем тексты на чанки для лучшего распознавания запроса

In [ ]:
def preprocess_documents(df):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunks = []  # Инициализация списка для хранения обработанных частей текста
    original_chunks = []  # Инициализация списка для хранения оригинальных частей текста
    for entry in df:
        title = entry['filename']  # имя файла
        paragraphs = entry['paragraphs']  # текст
        split_chunks = text_splitter.split_text(paragraphs)  # Разделение абзацев на части
        for chunk in split_chunks:
            cleaned_chunk = clean_text(chunk)  # Очистка текста
            expanded_chunk = expand_with_synonyms(cleaned_chunk)  # Расширение текста с помощью синонимов
            chunks.append((title, expanded_chunk))  # Добавление кортежа (название файла, обработанный текст) в список
            original_chunks.append((title, chunk))  # Добавление кортежа (название файла, оригинальный текст) в список
    return chunks, original_chunks

##Обучение векторизаторов для каждого отдела

### Функция векторизации TF-IDF

In [15]:
def create_and_fit_vectorizer(chunks):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(chunk for _, chunk in chunks)  # Применение векторизатора к чанкам
    return vectorizer, X, chunks  # Возврат векторизатора, матрицы TF-IDF и списка чанков

### Функция векторизации для каждого отдела

In [ ]:
def model_wrapper(df, model_type):
    documents = df[['filename', 'paragraphs']].to_dict(orient='records')
    chunks = preprocess_documents(documents) # Предобработка
    vectorizer, X, chunks = create_and_fit_vectorizer(chunks) # Обучение TF-IDF векторизатора
    return vectorizer, X, chunks     # Возврат векторизатора, матрицы TF-IDF и списка чанков

## Проверка работы модели

In [ ]:
#функция обработки запроса из набора тестовых запросов
def execute_queries(df, model_type, vectorizer, X, chunks):
    for item in knowledge_base["вопросы"]:
        if item['отдел'] == model_type:
            query = item['вопрос']  # Получаем текст запроса
            cleaned_query = clean_text(query)  # Очищаем и лемматизируем запрос
            expanded_query = expand_with_synonyms(cleaned_query)  # Расширяем запрос синонимами

            # Векторизуем расширенный запрос
            query_vec = vectorizer.transform([expanded_query])

            # Вычисляем косинусное расстояние между запросом и чанками
            results = cosine_similarity(X, query_vec).flatten()

            # Получаем индексы с наивысшей схожестью
            top_indices = results.argsort()[-5:][::-1]

            # Выводим информацию о запросе и результатах
            print("ДЕЛАЕМ ЗАПРОС: " + query)
            print("Материалы: " + str(item["материалы"]))
            print("Отдел: " + str(item["отдел"]))
            print()
            print("РЕЗУЛЬТАТЫ РАБОТЫ МОДЕЛИ: ")
            # Выводим топ-5 наиболее схожих записей
            for i in top_indices:
                similarity_score = results[i]
                print(f"Similarity: {similarity_score:.4f}")
                print(f"Title: {chunks[i][0]}")
                print(f"Text: {original_chunks[i][1]}")
                print()
        else:
            continue

# Работа модели и оценка результатов

## Обучение векторизаторов

In [ ]:
# Определяем список названий моделей
model_types = ['DRK', 'CMK', 'FTL', 'OTR', 'FT']

# Определяем список датасетов
dfs = [df_DRK, df_CMK, df_FTL, df_OTR, df_FT]

In [ ]:
# Создаем словарь для хранения результатов обучения для каждой комбинации типа модели и объекта DataFrame
results = {}

# Итерируемся по каждому объекту DataFrame и типу модели
for df, model_type in zip(dfs, model_types):
    # Вызываем функцию-обертку для модели, чтобы получить векторизатор, матрицу X и чанки
    vectorizer, X, chunks = model_wrapper(df, model_type)

    # Сохраняем результаты в словаре
    results[(model_type, id(df))] = {
        'vectorizer': vectorizer,  # Векторизатор
        'X': X,  # Матрица признаков X
        'chunks': chunks  # Чанки
    }

## Проверка работы векторизаторов

### Тестовые данные

In [16]:
# список текстовых запросов с эталонными документами
knowledge_base = {
    "вопросы": [
        {
            "вопрос": "Как быть если заказчик требует водителя с мед книжкой?",
            "материалы": "На забор груза нужен водитель с мед. книжкой.html",
            "отдел": "DRK"
        },
        {
            "вопрос": "Что входит в доставку в гипермаркеты?",
            "материалы": "Почему тариф на доставку в гипермаркет больше, чем стандартный.html",
            "отдел": "DRK"
        },
        {
            "вопрос": "Как занести контакт в черный список?",
            "материалы": "Клиент просит отключить уведомления (E-mail; ВК; Viber; СМС; Автообзвон; Обзвон).html",
            "отдел": "DRK"
        },
        {
            "вопрос": "Доступна ли перевозка грузов в Киргизию?",
            "материалы": "Курс лекций - База знаний ДРК - База знаний.html",
            "отдел": "DRK"
        },
        {
            "вопрос": "Как найти груз по номеру интернет заказа?",
            "материалы": "Выставление счета в Информации по грузу.html",
            "отдел": "DRK"
        },
        {
            "вопрос": "Как изменить страховую сумму при оформлении груза?",
            "материалы": "Оформление груза в СПО Пегас.html",
            "отдел": "DRK"
        },
        {
            "вопрос": "Возможна ли доставка день в день? Какие интервалы доставки?",
            "материалы": "Планирование доставки - База знаний ДРК - База знаний.html",
            "отдел": "DRK"
        },
        {
            "вопрос": "Что делать, если клиент предоставил новый номер для оповещения?",
            "материалы": "Оповещение о прибытии груза (без доставки) - База знаний ДРК - База знаний.html",
            "отдел": "DRK"
        },
        {
            "вопрос": "Почему не выходят на печать чеки об оплате и ПКО не отображается в реестре?",
            "материалы": "Проверка и проведение оплаты в СПО - База знаний ДРК - База знаний.html",
            "отдел": "DRK"
        },
        {
            "вопрос": "Как клиент может изменить дату авизации в личном кабинете?",
            "материалы": "Личный кабинет клиента (инструкция по применению) - База знаний ДРК - База знаний.html",
            "отдел": "DRK"
        },
        {
            "вопрос": "Клиент спрашивает, для чего мы запрашиваем письма от клиента, при предоставлении ему очередного спец условия",
            "материалы": "Заведение нового клиента и создание спец. условий - База знаний ДРК - База знаний.html",
            "отдел": "DRK"
        },
        {
            "вопрос": "Что написать клиенту, который просит полный доступ к личному кабинету?",
            "материалы": "Шаблоны ответов при обработке почты lk@pecom.ru - База знаний ДРК - База знаний.html",
            "отдел": "DRK"
        },
        {
            "вопрос": "Какая упаковка используется при перевозке автомобильных стекол?",
            "материалы": "Перечень грузов, подлежащих обязательной дополнительной упаковке - CMK - База знаний.html",
            "отдел": "CMK"
        },
        {
            "вопрос": "Сколько времени может проработать подметальная машина?",
            "материалы": "Альбом _Оборудование и инструменты, используемые на участках склада_ - CMK - База знаний.html",
            "отдел": "CMK"
        },
        {
            "вопрос": "Что делать, если клиент отказывается получения сопроводительных документов?",
            "материалы": "Инструкция пользователя «Организация перевозки сопроводительных документов» - CMK - База знаний.html",
            "отдел": "CMK"
        },
        {
           "вопрос": "Какие есть размеры паллетных рам?",
           "материалы": "Альбом _Материалы, используемые на участках склада_ - CMK - База знаний.html",
           "отдел": "CMK"
        },
        {
           "вопрос": "Как осуществить Контроль выполнения НТМЦ?",
           "материалы": "Инструкция пользователя _Создание набора ТМЦ при помощи РТСД_ - CMK - База знаний.html",
           "отдел": "CMK"
        },
        {
           "вопрос": "При отправке грузов в Минск прямые или транзитные рейсы?",
           "материалы": "GLOBAL - Продуктовая книга - База знаний.html",
           "отдел": "CMK"
        },
        {
           "вопрос": "Какие документы нужны для выдачи грузов в Беларусь (для физ. и юр. лиц)?",
           "материалы": "GLOBAL - Продуктовая книга - База знаний.html",
           "отдел": "CMK"
        },
        {
           "вопрос": "Что делать, если клиент выбрал способ подписания Договорных документов на бумаге?",
           "материалы": "Инструкция пользователя «Размещение Договорных документов с Перевозчиком в интерфейсе ПО «Договоры с контрагентами» - CMK - База знаний.html",
           "отдел": "CMK"
        },
        {
           "вопрос": "Как провести обработку упрощенной выдачи по нескольким грузам?",
           "материалы": "Инструкция пользователя _Оформление выдачи груза в СПО_ - CMK - База знаний.html",
           "отдел": "CMK"
        },
        {
           "вопрос": "Когда следует использовать нестандартную защитную упаковку?",
           "материалы": "Стандарт «Дополнительная упаковка груза» - CMK - База знаний.html",
           "отдел": "CMK"
        },
        {
           "вопрос": "Как следует грузить негабаритный груз?",
           "материалы": "Технологическая инструкция _Погрузка и разгрузка ТС с использованием КВП_ - CMK - База знаний.html",
           "отдел": "CMK"
        },
        {
           "вопрос": "Что включает общий вид карточки контрагента?",
           "материалы": "Карточка контрагента - MVP - Full Truck Load - База знаний.html",
           "отдел": "FTL"
        },
        {
           "вопрос": "Как загрузить прайс лист по услугам FTL?",
           "материалы": "Загрузка прайса по услугам FTL в 1С_ TMS Логистика - в проде - Full Truck Load - База знаний.html",
           "отдел": "FTL"
        },
        {
           "вопрос": "Что включает отчет по агентским вознаграждениям FTL?",
           "материалы": "Отчет по агентским вознаграждениям - MVP - Full Truck Load - База знаний.html",
           "отдел": "FTL"
        },
        {
           "вопрос": "Как поступить, если у заявки выставлен статус 'Подтверждение заявки с клиентом'?",
           "материалы": "Алгоритм действий сотрудников по работе с заявкой FTL в 1C_TMS (без документа Рейс) - в проде, версия от 21 года(возможно не актуально) - Full Truck Load - База знаний.html",
           "отдел": "FTL"
        },
        {
           "вопрос": "Как создать новую географическую зону?",
           "материалы": "Редактирование географических зон - Full Truck Load - База знаний.html",
           "отдел": "FTL"
        },
        {
           "вопрос": "Какие бизнес правила страхования актуальны сегодня?",
           "материалы": "Услуги по страхованию груза и срока при перевозке груза по территории РФ и за ее пределами - MVP - Full Truck Load - База знаний.html",
           "отдел": "FTL"
        },
        {
           "вопрос": "Кто изменял документ РасчитатьСпецУсловие_ЗаменаУслуг?",
           "материалы": "ОТР (RND-829).html",
           "отдел": "OTR"
        },
        {
           "вопрос": "Кто наполняет таблицу в базе данных api.RawLog?",
           "материалы": "ОТР (DARCH-1059).html",
           "отдел": "OTR"
        },
        {
           "вопрос": "Когда создана форма документа споВосстановлениеМДИ?",
           "материалы": "ОТР (DBUH-1198).html",
           "отдел": "OTR"
        },
        {
           "вопрос": "Как реализован механизм многопоточной обработки ЭТН?",
           "материалы": "ОТР (DED-51).html",
           "отдел": "OTR"
        },
        {
           "вопрос": "Кто на форму Обработка.ЗачетПереплат добавлял элементы ПериодВыставленияАктов и КонтрагентыНачинаютсяССимволов?",
           "материалы": "ОТР (DPEGAS-32409).html",
           "отдел": "OTR"
        },
        {
           "вопрос": "Что произойдет с просроченными грузами в ПВЗ?",
           "материалы": "ФТ (EPIC-11332).html",
           "отдел": "FT"
        },
        {
           "вопрос": "Возможно ли создание большего количества характеров груза, чем в документе 'Настройка матрица соответствия внутреннего груза'?",
           "материалы": "ФТ (EPIC-10470).html",
           "отдел": "FT"
        },
        {
           "вопрос": "Выставление документа Счет на оплату клиенту, при изменении данных о плательщике, городе оплаты и сумме?",
           "материалы": "ФТ (EPIC-10552).html",
           "отдел": "FT"
        },
        {
           "вопрос": "Возможно ли изменить возвратные данные по грузу, до того, как груз станет возвратным?",
           "материалы": "ФТ (EPIC-10142).html",
           "отдел": "FT"
        },
        {
           "вопрос": "Была ли доработка документа 'Инвентаризация на отправку'?",
           "материалы": "ФТ (EPIC-9495).html",
           "отдел": "FT"
        },
        {
           "вопрос": "Была ли доработка документа 'Приемка'?",
           "материалы": "ФТ (EPIC-13571).html",
           "отдел": "FT"
        }
    ]
}


### Проверка результатов выдачи

In [18]:
# Итерируемся по каждому типу модели и соответствующему ей объекту DataFrame
for model_type, df in zip(model_types, dfs):
    # Получаем соответствующий векторизатор, X и chunks на основе типа модели
    if model_type == 'DRK':
        vectorizer, X, chunks = results[('DRK', id(df_DRK))]['vectorizer'], results[('DRK', id(df_DRK))]['X'], results[('DRK', id(df_DRK))]['chunks']
        execute_queries(df, model_type, vectorizer, X, chunks) #обрабатываем каждый запрос в тестовых данных в зависимости от отдела и выводим результаты
    elif model_type == 'CMK':
        vectorizer, X, chunks = results[('CMK', id(df_CMK))]['vectorizer'], results[('CMK', id(df_CMK))]['X'], results[('CMK', id(df_CMK))]['chunks']
        execute_queries(df, model_type, vectorizer, X, chunks)
    elif model_type == 'FTL':
        vectorizer, X, chunks = results[('FTL', id(df_FTL))]['vectorizer'], results[('FTL', id(df_FTL))]['X'], results[('FTL', id(df_FTL))]['chunks']
        execute_queries(df, model_type, vectorizer, X, chunks)
    elif model_type == 'OTR':
        vectorizer, X, chunks = results[('OTR', id(df_OTR))]['vectorizer'], results[('OTR', id(df_OTR))]['X'], results[('OTR', id(df_OTR))]['chunks']
        execute_queries(df, model_type, vectorizer, X, chunks)
    elif model_type == 'FT':
        vectorizer, X, chunks = results[('FT', id(df_FT))]['vectorizer'], results[('FT', id(df_FT))]['X'], results[('FT', id(df_FT))]['chunks']
        execute_queries(df, model_type, vectorizer, X, chunks)
    else:
        continue

ДЕЛАЕМ ЗАПРОС: Как быть если заказчик требует водителя с мед книжкой?
Материалы: На забор груза нужен водитель с мед. книжкой.html
Отдел: DRK

РЕЗУЛЬТАТЫ РАБОТЫ МОДЕЛИ: 
Similarity: 0.4355
Title: На забор груза нужен водитель с мед. книжкой

Similarity: 0.2283
Title: Планирование забора - База знаний ДРК - База знаний

Similarity: 0.1873
Title: Заказчик в печатной форме Поручения экспедитору

Similarity: 0.1706
Title: Планирование забора - База знаний ДРК - База знаний

Similarity: 0.1501
Title: Личный кабинет клиента (инструкция по применению) - База знаний ДРК - База знаний

ДЕЛАЕМ ЗАПРОС: Что входит в доставку в гипермаркеты?
Материалы: Почему тариф на доставку в гипермаркет больше, чем стандартный.html
Отдел: DRK

РЕЗУЛЬТАТЫ РАБОТЫ МОДЕЛИ: 
Similarity: 0.5627
Title: Почему тариф на доставку в гипермаркет больше, чем стандартный

Similarity: 0.2734
Title: Скрипты для чатов - База знаний ДРК - База знаний

Similarity: 0.2586
Title: Курс лекций - База знаний ДРК - База з